# Identifying Arbitrage Opportunities

This notebook is for testing the individual functions within lambda_script.py, outside of a lambda.

This script identifies arbitrage opportunities across a set of cryptocurrency exchanges by returning the current value of any given cryptocurrency on any given exchange, and identifying the exchanges with the highest and lowest values for said cryptocurrency.

## Setup

In [27]:
# Imports
import json
import operator
import requests
from datetime import datetime

In [28]:
# Declare variables
currency1 = 'BTC'
currency2 = ['USD', 'USDT']

exchanges = [
    "Bitfinex",
    "Binance",
    "Poloniex",
    "Coinbase",
    "BitTrex" 
]

api_key = 'c32565db2e025bfaba2a25ccc2068d5aa44ddaefd51277bfd833e5649606defc'

## Define Functions

In [31]:
def get_coin_price(currency1, currency2, exchange, api_key):
    url = 'https://min-api.cryptocompare.com/data/price?fsym={}&tsyms={}&api_key={}&e={}'.format(
        currency1, 
        currency2, 
        api_key, 
        exchange
    )
    
    headers = {
        'Content-Type': "application/json"
    }
    response = requests.request("GET", url, headers=headers).json()
    
    return response


In [32]:
def find_arbitarage_opportunities(currency1, currency2, exchanges, api_key):
    
    usd_exchange = ['Coinbase', 'BitTrex', 'Bitfinex']
    usdt_exchange = ['Binance', 'Poloniex']
    exchange_list = []
    value_list = []
    
    # create price list
    for exchange in exchanges:
        if exchange in usd_exchange:
            coin_price = get_coin_price(currency1, currency2[0], exchange, api_key)['USD']
        else:
            coin_price = get_coin_price(currency1, currency2[1], exchange, api_key)['USDT']
        value_list.append(coin_price)
        
    # create exchange list
    for i in range(0, len(exchanges)):
        exchange_list.append(exchanges[i])
        
    # create dictionary
    response = dict(zip(exchange_list, value_list))
    
    # min and max
    max_value = max(response.values())
    max_exchange = max(response.items(), key=operator.itemgetter(1))[0]
    min_value = min(response.values())
    min_exchange = min(response.items(), key=operator.itemgetter(1))[0]
    
    # time
    now = datetime.now()
    timestamp = str(now.month) + '-' + str(now.day) + '-' + str(now.year) + ' / ' + str(now.hour) + ':' + str(now.minute) + ':' + str(now.second)
    
    # gains
    dollar_dif = round(max_value - min_value)
    percent_gain = ((max_value - min_value)/min_value)*100
    
    # create output
    output = create_output(
        response, 
        max_value, 
        max_exchange,
        min_value, 
        min_exchange,
        timestamp,
        dollar_dif,
        percent_gain
    )
    
    return output

In [33]:
def create_output(output, max_value, max_exchange, min_value, min_exchange, timestamp, dollar_dif, percent_gain):
    output = {
        'BTC': output,
        'max_exchange': max_exchange,
        'max_value': max_value,
        'min_exchange': min_exchange,
        'min_value': min_value,
        'timestamp': timestamp,
        'dollar_dif': dollar_dif,
        'percent_gain': percent_gain
    }
    
    return output

In [34]:
response = find_arbitarage_opportunities(currency1, currency2, exchanges, api_key)

In [35]:
response

{'BTC': {'Binance': 7756.06,
  'BitTrex': 7892.13,
  'Bitfinex': 7753.11,
  'Coinbase': 7880.66,
  'Poloniex': 7814.11},
 'dollar_dif': 139,
 'max_exchange': 'BitTrex',
 'max_value': 7892.13,
 'min_exchange': 'Bitfinex',
 'min_value': 7753.11,
 'percent_gain': 1.793086903191112,
 'timestamp': '5-13-2019 / 11:51:36'}